**MODEL 2**

**2. Meeting Summarizer + Action Tracker for Indian Teams**

In [ ]:
# 🔄 End-to-End Meeting Assistant

# ✅ Step 1: Install required libraries
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q requests fpdf cryptography


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import whisper
import os
import json
import re
import requests
import shutil
from fpdf import FPDF
from cryptography.fernet import Fernet
from google.colab import files
from IPython.display import display, Audio

In [ ]:
# ✅ Step 2: Upload audio file
print("⬆️ Upload your meeting audio file (.mp3/.wav)")
uploaded = files.upload()
audio_file = list(uploaded.keys())[0]

⬆️ Upload your meeting audio file (.mp3/.wav)


Saving sample2_audio.mp3 to sample2_audio.mp3


In [ ]:
# ✅ Step 3: Transcribe using Whisper
model = whisper.load_model("base")
print(f"\U0001F50A Transcribing: {audio_file} ...")
result = model.transcribe(audio_file)
transcript = result['text']

🔊 Transcribing: sample2_audio.mp3 ...


In [ ]:
# ✅ Step 4: Display transcript
print("\n\U0001F4DD Transcript:")
print(transcript)


📝 Transcript:
 Abyshek to documents Sprint delays by Friday. A deedy to build automation prototype by Thursday. Set new weekly chickpeo. Doodle poll to follow.


In [ ]:
# ✅ Step 5: Rule-based Task Extractor (Fallback if LLM not used)
def extract_tasks(transcript):
    lines = transcript.split(". ")
    tasks = []
    for line in lines:
        if re.search(r'(?i)\b(should|must|need to|assign|complete|by|deadline|responsible for|follow up|send|update)\b', line):
            tasks.append(line.strip())
    return tasks

In [ ]:
# ✅ Step 6: Summary + Tasks (LLM fallback to rule-based)
MISTRAL_API_KEY = "YOUR_MISTRAL_API_KEY"
MISTRAL_API_URL = "https://api.mistral.ai/v1/chat/completions"

In [ ]:
import re
import requests

def generate_summary_and_tasks(transcript_text):
    try:
        headers = {
            "Authorization": f"Bearer {MISTRAL_API_KEY}",
            "Content-Type": "application/json"
        }
        prompt = f"""
    Act as an AI meeting assistant. Your task is to deeply analyze the provided transcript of a multi-lingual meeting (could be in English, Hindi, or Tamil) and extract structured insights. Return the following:

        1. **Concise Summary**: A 3-4 line overview highlighting the main discussion themes.
        2. **Action Points**:
            - Bullet points listing key tasks
            - Include responsible individuals (if mentioned)
            - Add deadlines or due dates (if mentioned)
            - Group them by categories if possible (e.g., technical, admin, client-related)
        3. **Meeting Sentiment (optional)**: Was the tone collaborative, urgent, confused, etc.?
        4. **Critical Decisions Taken** (if any): Mention any conclusions or approvals.

        Transcript:
        {transcript_text}
        """

        data = {
            "model": "mistral-medium",
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "temperature": 0.3
        }

        response = requests.post(MISTRAL_API_URL, headers=headers, json=data)
        result = response.json()
        return result['choices'][0]['message']['content']
    except Exception as e: # Catching a broader exception to handle potential API errors
        print(f"Error during LLM processing: {e}") # Print the actual error
        return "\n\nRule-based Tasks:\n" + "\n".join(extract_tasks(transcript_text))

print("\n\U0001F9D0 Generating summary and tasks...")
output = generate_summary_and_tasks(transcript) # Changed transcription to transcript
print("\n✅ Summary + Tasks:")
print(output)


🧐 Generating summary and tasks...
Error during LLM processing: Expecting value: line 1 column 1 (char 0)

✅ Summary + Tasks:


Rule-based Tasks:
Abyshek to documents Sprint delays by Friday
A deedy to build automation prototype by Thursday


In [ ]:
# ✅ Step 7: Save to TXT, PDF, CSV
with open("meeting_summary_tasks.txt", "w", encoding="utf-8") as f:
    f.write("Transcript:\n" + transcript + "\n\n")
    f.write("Summary + Tasks:\n" + output)

In [ ]:
# Save as CSV
with open("meeting_summary_tasks.csv", "w", encoding="utf-8") as f:
    f.write("Task\n")
    for line in output.split("\n"):
        if line.strip().startswith("-"):
            f.write(line.strip("- ").strip() + "\n")

In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=6fb7d6d64da21dbc812752e7e78a176aca66d6a4b8f456630735ccd024bce7ff
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [ ]:
# Save as PDF
from fpdf import FPDF
from google.colab import files

pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.multi_cell(0, 10, "Transcript:\n" + transcript + "\n\nSummary + Tasks:\n" + output)
pdf.output("meeting_summary_tasks.pdf")

files.download("meeting_summary_tasks.txt")
files.download("meeting_summary_tasks.csv")
files.download("meeting_summary_tasks.pdf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install cryptography

In [ ]:
# ✅ Step 8: Encrypt Output Files
from cryptography.fernet import Fernet
import os

key = Fernet.generate_key()
cipher = Fernet(key)

for file_name in ["meeting_summary_tasks.txt", "meeting_summary_tasks.csv", "meeting_summary_tasks.pdf"]:
    with open(file_name, "rb") as f:
        encrypted_data = cipher.encrypt(f.read())
    with open(file_name + ".enc", "wb") as ef:
        ef.write(encrypted_data)
    os.remove(file_name)  # Delete original

print("\n🔐 Files encrypted using AES-256 equivalent (Fernet). Key:")
print(key.decode())


🔐 Files encrypted using AES-256 equivalent (Fernet). Key:
QYRN7r3HBFUf19CyHwCOj5gEZrtwtQEVCGFzO76qW6Q=


In [ ]:
# ✅ Step 9: Delete Temp Cache
if os.path.exists(audio_file):
    os.remove(audio_file)
print("\n❌ Audio file deleted from cache.")


❌ Audio file deleted from cache.


**A smart meeting assistant system that:

Records live or uploaded meetings

Transcribes audio (speech-to-text)

Summarizes the content (e.g., meeting minutes)

Extracts tasks and deadlines

Sends summaries and tasks to users via their preferred platforms

✅ It helps teams avoid manual note-taking, keeps meetings actionable, and improves follow-up.

**

📤 Export Options
✅ Summary + Tasks saved as .txt, .csv, and .pdf

✅ Auto-download in Colab

🔒 Security & Privacy
✅ AES-256 encryption (Fernet)

✅ Cache audio auto-deletion after processing

✅ Encrypted file output with downloadable key

🧠 Improved Task Extraction
✅ Fallback rule-based extractor using regex if LLM API fails

🖥️ Offline Support Acknowledgement
✅ Note included about using small/offline models (like TinyLlama) for summary generation in local environments


